In [1]:
# v1.0.7 - 14/02/2020

import sys, os
sys.path.append('../') # to allow loading internal libraries from other people/projects
sys.path.append('../../../fc00232/notebooks/utils_fc00232/') # to allow loading Hector library
import time, datetime, math

import glob
import re
from imp import reload
from collections import Counter
from unidecode import unidecode

import numpy as np
import pandas as pd
pd.set_option("display.max_columns",1000)
pd.set_option("display.max_rows",1000)
import geopandas as gpd

from shapely.geometry import Point
crs = {"init":"epsg:4326"}

# ----------------------------------------------------------------------------------------------------------

import matplotlib.pyplot as plt
plt.style.use("ggplot") # to make matplotlib look better
%matplotlib inline
import seaborn as sns

import plotly
import chart_studio
import cufflinks as cf
cf.set_config_file(offline=True) # to make cufflinks work offline

import pandas_bokeh
pandas_bokeh.output_notebook()

#from plotly import colors.DEFAULT_PLOTLY_COLORS # to know default plotly colors
from plotly import graph_objs as go
BKCF_colors = ['#e32592','#fae700','#34b233','#58585a','#939597','#E0E1DD','#5bcbf5','#fe4540','#c7be71'] # to use with cufflinks
layout = go.Layout(colorway=BKCF_colors) # to use with plain plotly
from plotly import tools as tls
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) # to allow plotly offline for jupyter notebooks
from bokeh.plotting import save # to allow saving pandas bokeh plots into html files
import plotly.express as px
import yfinance as yf
from datetime import date
import calendar
# ----------------------------------------------------------------------------------------------------------

from IPython.core.display import display
#display(HTML("<style>.container { width:95% !important; }</style>")) # to make jupyter notebook appearence bigger

# ----------------------------------------------------------------------------------------------------------

if "HOME" in os.environ:
    from spark_conn_o import get_connection
    spark = get_connection("yarn-client","Carga JSONs Catastro","fc00257","25","5G","4","BKCF")
    sc = spark.sparkContext

# ----------------------------------------------------------------------------------------------------------
import pickle
from urllib.request import urlopen
import json
#from j_library import data_anl as danl
#from j_library import utils as u
#from pyspark import StorageLevel
#from pyspark.sql import functions as F, types as T, Row, Window, SparkSession

#import lib_auxiliar_clas_binaria as lib
#from dython import nominal as dynom

# ----------------------------------------------------------------------------------------------------------

#import findspark
#findspark.init()



#spark = SparkSession.builder.getOrCreate()
#sc = spark.sparkContext

Loading BokehJS ...

In [2]:
#Save in pickle format
def save_obj(obj, name, path="./obj"):
    if not(os.path.isdir(path)):
        os.mkdir(path)
        print("Creado directorio './obj' donde guardar objeto")
    with open("{}/{}.pkl".format(path,name), "wb") as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
#Load a pickle        
def load_obj(name, path="./obj"):
    with open("{}/{}.pkl".format(path, name), "rb") as f:
        return pickle.load(f)
    
#Download json
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

def plot_metrica(df, metrica):
    df = df[['symbol','date','{0}'.format(metrica)]]
    colors = ['orange', 'pink','purple','brown','blue','green','black','grey','yellow','azure','gold']
    symbols = df['symbol'].drop_duplicates().values.tolist()
    if len(symbols)>12:
        print("Introduce como máximo 12 compañías")
    else:
        fig = px.line(df[df['symbol']==symbols[0]], x = df.columns[1], y= df.columns[2], color = 'symbol',
                      color_discrete_sequence=['red'],#line_dash_sequence=['dash'], 
                      title="{0}".format(metrica))
        for i in range(1,len(symbols)):
            fig1 = px.line(df[df['symbol']==symbols[i]], x = df.columns[1], y = df.columns[2], color = 'symbol',
                           color_discrete_sequence=[colors[i-1]])
            fig.add_trace(fig1.data[0])
        return fig
    
from ipywidgets import interact, IntSlider
from IPython.display import display

def freeze_header(df, num_rows=300, num_columns=10, step_rows=1,step_columns=1):
    """
    Freeze the headers (column and index names) of a Pandas DataFrame. A widget
    enables to slide through the rows and columns.

    Parameters
    ----------
    df : Pandas DataFrame
        DataFrame to display
    num_rows : int, optional
        Number of rows to display
    num_columns : int, optional
        Number of columns to display
    step_rows : int, optional
        Step in the rows
    step_columns : int, optional
        Step in the columns

    Returns
    -------
    Displays the DataFrame with the widget
    """
    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]),
                                 max=df.shape[0],
                                 step=step_rows,
                                 description='rows',
                                 readout=False,
                                 disabled=False,
                                 continuous_update=True,
                                 orientation='horizontal',
                                 slider_color='purple'),
              last_column=IntSlider(min=min(num_columns, df.shape[1]),
                                    max=df.shape[1],
                                    step=step_columns,
                                    description='columns',
                                    readout=False,
                                    disabled=False,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    slider_color='purple'))
    def _freeze_header(last_row, last_column):
        display(df.iloc[max(0, last_row-num_rows):last_row,
                        max(0, last_column-num_columns):last_column])

In [3]:
#Big tech
amzn = load_obj("AMZN")
netflix = load_obj("NFLX")
msft = load_obj("MSFT")
aapl = load_obj("AAPL")
adobe = load_obj("ADBE")
crm = load_obj("CRM")
oracle = load_obj("ORCL")
cisco = load_obj("CSCO")
ibm = load_obj("IBM")
facebook = load_obj("FB")
google = load_obj("GOOG")

#Small tech
alteryx = load_obj("AYX")
ttd = load_obj("TTD")
roku = load_obj("ROKU")
telaria = load_obj("TLRA")
arista = load_obj("ANET")
shopify = load_obj("SHOP")
square = load_obj("SQ")
paycom = load_obj("PAYC")
service_now = load_obj("NOW")
twitter = load_obj("TWTR")
dropbox = load_obj("DBX")

#Others interesting
paypal = load_obj("PYPL")
visa = load_obj("V")
mastercard = load_obj("MA")
dominos = load_obj("DPZ")
chipotle = load_obj("CMG")

#Computer hardware
xerox = load_obj("XRX")
texas_instruments = load_obj("TXN")
nvidia = load_obj("NVDA")
intel = load_obj("INTC")
amd = load_obj("AMD")
hp = load_obj("HPQ")

#Healthcare & farmacy & biotech
pfizer = load_obj("PFE")
bristol_myers = load_obj("BMY")
merck =load_obj("MRK")
abiomed = load_obj("ABMD")
veeva = load_obj("VEEV")

#Stalwarts
procter = load_obj("PG")
disney = load_obj("DIS")
jj = load_obj("JNJ")
mc = load_obj("MCD")
walmart = load_obj("WMT")
coke = load_obj("KO")
exxon = load_obj("XOM")
starbucks = load_obj("SBUX") 
jpmorgan = load_obj("JPM")
home_depot = load_obj("HD")
lowes = load_obj("LOW")
target = load_obj("TGT")
costo = load_obj("COST")
adp = load_obj("ADP")
general_dynamics = load_obj("GD")
pepsico = load_obj("PEP")
colgate = load_obj("CL")
kellog = load_obj("K")
general_mills = load_obj("GIS")
kimberly_clark = load_obj("KMB")

#Cyclicals
ford = load_obj("F")
marriott = load_obj("MAR")
southwest = load_obj("LUV")
delta_airlines = load_obj("DAL")
wendys = load_obj("WEN")
yum_brands = load_obj("YUM")


#Bad looking
the_limited = load_obj("LB")
blackberry = load_obj("BB")
nokia = load_obj("NOK")
kodak = load_obj("KODK")
gap = load_obj("GPS")
boeing = load_obj("BA")
harley_davidson =load_obj("HOG")
philip_morris = load_obj("MO")
general_electric = load_obj("GE")
macys = load_obj("M")

#Oil companies
occidental_petroleum = load_obj("OXY")
apache = load_obj("APA")
marathon_oil = load_obj("MRO")
valaris = load_obj("VAL")
tullow_oil = load_obj("TLW.L")
schulmberger = load_obj("SLB")

# Company profile

In [4]:
companies = ['AAPL','TLW.L','FB','K']

In [5]:
def profile(companies):
    link = ("https://financialmodelingprep.com/api/v3/company/profile/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link)).reset_index()
    simbolo = df_final['symbol'][1]
    df_final = df_final[['index','profile']] 
    df_final.index = df_final['index']
    df_final = df_final.T.reset_index()
    df_final = df_final.iloc[1:]
    df_final['symbol'] = simbolo
    df_final = df_final[['symbol','companyName','exchange','mktCap','price','range','lastDiv','beta','ceo','industry','sector','volAvg']]
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/company/profile/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link)).reset_index()
        if df.shape[0]!=0:
            simbolo = df['symbol'][1]
            df= df[['index','profile']] 
            df.index = df['index']
            df = df.T.reset_index()
            df = df.iloc[1:]
            df['symbol'] = simbolo
            df = df[['symbol','companyName','exchange','mktCap','price','range','lastDiv','beta','ceo','industry','sector','volAvg']]
            df_final = pd.concat([df_final, df])
    return df_final  

In [6]:
profile(companies)

index,symbol,companyName,exchange,mktCap,price,range,lastDiv,beta,ceo,industry,sector,volAvg
1,AAPL,Apple Inc.,Nasdaq Global Select,1.13513071E12,259.43,170.27-327.85,2.92,1.228499,Timothy D. Cook,Computer Hardware,Technology,49632122
1,FB,Facebook Inc.,Nasdaq Global Select,4.81241465E11,168.83,137.1-224.2,0,1.062394,Mark Zuckerberg,Online Media,Technology,21727493
1,K,Kellogg Company,New York Stock Exchange,2.14233129E10,62.55,51.34-71.05,2.24,0.530354,Steven A. Cahillane,Consumer Packaged Goods,Consumer Defensive,2513352


# Annual and quarter financial statements

In [7]:
#Annual income statement
def income_annual(companies):
    link = ("https://financialmodelingprep.com/api/v3/financials/income-statement/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['financials'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financials/income-statement/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['financials'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Quarter income statement
def income_quarter(companies):
    link = ("https://financialmodelingprep.com/api/v3/financials/income-statement/{0}?period=quarter".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['financials'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financials/income-statement/{0}?period=quarter".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['financials'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Annual balance sheet
def balance_annual(companies):
    link = ("https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['financials'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['financials'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Quarter balance sheet
def balance_quarter(companies):
    link = ("https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{0}?period=quarter".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['financials'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{0}?period=quarter".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['financials'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Annual cashflow statement
def cashflow_annual(companies):
    link = ("https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['financials'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['financials'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Quarter cashflow statement
def cashflow_quarter(companies):
    link = ("https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['financials'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{0}?period=quarter".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['financials'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

In [8]:
display(income_annual(companies).head())
display(balance_annual(companies).head())
display(cashflow_annual(companies).head())

,symbol,date,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin
0,AAPL,2019-09-28,2.60174e+11,-0.0204107758053,1.61782e+11,98392000000.0,16217000000.0,18245000000.0,34462000000.0,63930000000.0,3576000000.0,65737000000.0,10481000000.0,0.0,0.0,55256000000.0,0.0,55256000000.0,11.97,11.89,4617834000.0,4648913000.0,3.03705403822,0.37817768109,0.293945590259,0.245720171885,0.21238094506,0.226371582095,76477000000.0,63930000000.0,55256000000.0,0.252665523842,0.21238094506
1,AAPL,2018-09-29,2.65595e+11,0.158619576503,1.63756e+11,1.01839e+11,14236000000.0,16705000000.0,30941000000.0,70898000000.0,3240000000.0,72903000000.0,13372000000.0,0.0,0.0,59531000000.0,0.0,59531000000.0,12.01,11.91,4955377000.0,5000109000.0,2.74234021698,0.3834371882,0.307991490804,0.266940266195,0.224142020746,0.24142397259,81801000000.0,70898000000.0,59531000000.0,0.274489354092,0.224142020746
2,AAPL,2017-09-30,2.29234e+11,0.063045181994,1.41048e+11,88186000000.0,11581000000.0,15261000000.0,26842000000.0,61344000000.0,2323000000.0,64089000000.0,15738000000.0,0.0,0.0,48351000000.0,0.0,48351000000.0,9.27,9.21,5217242000.0,5251692000.0,2.43140686849,0.384698604919,0.311912718009,0.267604282087,0.210924208451,0.223121351981,71501000000.0,61344000000.0,48351000000.0,0.279578945532,0.210924208451
3,AAPL,2016-09-24,2.15639e+11,-0.077342061913,1.31376e+11,84263000000.0,10045000000.0,14194000000.0,24239000000.0,60024000000.0,1456000000.0,61372000000.0,15685000000.0,0.0,0.0,45687000000.0,0.0,45687000000.0,8.35,8.31,5470820000.0,5500281000.0,2.20897805039,0.390759556481,0.327069778658,0.278354101067,0.211867983064,0.246198507691,70529000000.0,60024000000.0,45687000000.0,0.284605289396,0.211867983064
4,AAPL,2015-09-26,2.33715e+11,0.278563418037,1.40089e+11,93626000000.0,8067000000.0,14329000000.0,22396000000.0,71230000000.0,733000000.0,72515000000.0,19121000000.0,0.0,0.0,53394000000.0,0.0,53394000000.0,9.28,9.22,5753421000.0,5793069000.0,1.99566067658,0.400599020174,0.35293840789,0.304772907173,0.228457736987,0.299591382667,82487000000.0,71230000000.0,53394000000.0,0.310271056629,0.228457736987


,symbol,date,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,Tax assets,Total non-current assets,Total assets,Payables,Short-term debt,Total current liabilities,Long-term debt,Total debt,Deferred revenue,Tax Liabilities,Deposit Liabilities,Total non-current liabilities,Total liabilities,Other comprehensive income,Retained earnings (deficit),Total shareholders equity,Investments,Net Debt,Other Assets,Other Liabilities
0,AAPL,2019-09-28,48844000000.0,51713000000.0,1.00557e+11,22926000000.0,4106000000.0,1.62819e+11,37378000000.0,0.0,1.05341e+11,19640000000.0,1.75697e+11,3.38516e+11,46236000000.0,10260000000.0,1.05718e+11,91807000000.0,1.02067e+11,5522000000.0,15619000000.0,0.0,1.4231e+11,2.48028e+11,40730764000,45898000000.0,90488000000.0,1.57054e+11,53223000000.0,0.0,0.0
1,AAPL,2018-09-29,25913000000.0,40388000000.0,66301000000.0,23186000000.0,3956000000.0,1.31339e+11,41304000000.0,0.0,1.70799e+11,6610000000.0,2.34386e+11,3.65725e+11,55888000000.0,8784000000.0,1.16866e+11,93735000000.0,1.02519e+11,7543000000.0,9694000000.0,0.0,1.41712e+11,2.58578e+11,35446014000,70400000000.0,1.07147e+11,2.11187e+11,76606000000.0,0.0,0.0
2,AAPL,2017-09-30,20289000000.0,53892000000.0,74181000000.0,17874000000.0,4855000000.0,1.28645e+11,33783000000.0,8015000000.0,1.94714e+11,8974000000.0,2.46674e+11,3.75319e+11,49049000000.0,6496000000.0,1.00814e+11,97207000000.0,1.03703e+11,7548000000.0,8407000000.0,0.0,1.71962e+11,2.41272e+11,30740799000,98330000000.0,1.34047e+11,2.48606e+11,83414000000.0,0.0,0.0
3,AAPL,2016-09-24,20484000000.0,46671000000.0,67155000000.0,15754000000.0,2132000000.0,1.06869e+11,27010000000.0,8620000000.0,1.7043e+11,10015000000.0,2.14817e+11,3.21686e+11,37294000000.0,3500000000.0,79006000000.0,75427000000.0,78927000000.0,8080000000.0,7700000000.0,0.0,1.4045e+11,1.93437e+11,25914834000,96364000000.0,1.28249e+11,2.17101e+11,58443000000.0,0.0,0.0
4,AAPL,2015-09-26,21120000000.0,20481000000.0,41601000000.0,16849000000.0,2349000000.0,89378000000.0,22471000000.0,9009000000.0,1.64065e+11,5546000000.0,2.01101e+11,2.90479e+11,35490000000.0,2500000000.0,80610000000.0,53463000000.0,55963000000.0,8940000000.0,6900000000.0,0.0,1.14576e+11,1.71124e+11,21837247000,92284000000.0,1.19355e+11,1.84546e+11,34843000000.0,0.0,0.0


,symbol,date,Depreciation & Amortization,Stock-based compensation,Operating Cash Flow,Capital Expenditure,Acquisitions and disposals,Investment purchases and sales,Investing Cash flow,Issuance (repayment) of debt,Issuance (buybacks) of shares,Dividend payments,Financing Cash Flow,Effect of forex changes on cash,Net cash flow / Change in cash,Free Cash Flow,Net Cash/Marketcap
0,AAPL,2019-09-28,12547000000.0,6068000000.0,69391000000.0,10495000000.0,-624000000.0,-8804000000.0,45896000000.0,-8805000000.0,-66116000000.0,-14119000000.0,-90976000000.0,0.0,24311000000.0,58896000000.0,0.0
1,AAPL,2018-09-29,10903000000.0,5340000000.0,77434000000.0,13313000000.0,-721000000.0,-18375000000.0,16066000000.0,-6500000000.0,-72069000000.0,-13712000000.0,-87876000000.0,0.0,5624000000.0,64121000000.0,0.0
2,AAPL,2017-09-30,10157000000.0,4840000000.0,63598000000.0,12795000000.0,-329000000.0,-1520000000.0,-46446000000.0,-3500000000.0,-32345000000.0,-12769000000.0,-17347000000.0,0.0,-195000000.0,50803000000.0,0.0
3,AAPL,2016-09-24,10505000000.0,4210000000.0,65824000000.0,13548000000.0,-297000000.0,-9852000000.0,-45977000000.0,-2500000000.0,-29227000000.0,-12150000000.0,-20483000000.0,0.0,-636000000.0,52276000000.0,0.0
4,AAPL,2015-09-26,11257000000.0,3586000000.0,81266000000.0,11488000000.0,-343000000.0,-20715000000.0,-56274000000.0,-0.0,-34710000000.0,-11561000000.0,-17716000000.0,0.0,7276000000.0,69778000000.0,0.0


# Annual financial ratios

In [9]:
#Valuation ratios
def valuation_ratios(companies):
    link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['ratios'].apply(pd.Series)], axis=1)
    df_final = pd.concat([df_final[['symbol','date']],df_final['investmentValuationRatios'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['ratios'].apply(pd.Series)], axis=1)
            df = pd.concat([df[['symbol','date']],df['investmentValuationRatios'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Profitability ratios
def profitability_ratios(companies):
    link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['ratios'].apply(pd.Series)], axis=1)
    df_final = pd.concat([df_final[['symbol','date']],df_final['profitabilityIndicatorRatios'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['ratios'].apply(pd.Series)], axis=1)
            df = pd.concat([df[['symbol','date']],df['profitabilityIndicatorRatios'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Operating ratios
def operating_ratios(companies):
    link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['ratios'].apply(pd.Series)], axis=1)
    df_final = pd.concat([df_final[['symbol','date']],df_final['operatingPerformanceRatios'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['ratios'].apply(pd.Series)], axis=1)
            df = pd.concat([df[['symbol','date']],df['operatingPerformanceRatios'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Liquidity ratios
def liquidity_ratios(companies):
    link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['ratios'].apply(pd.Series)], axis=1)
    df_final = pd.concat([df_final[['symbol','date']],df_final['liquidityMeasurementRatios'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['ratios'].apply(pd.Series)], axis=1)
            df = pd.concat([df[['symbol','date']],df['liquidityMeasurementRatios'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Debt ratios
def debt_ratios(companies):
    link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['ratios'].apply(pd.Series)], axis=1)
    df_final = pd.concat([df_final[['symbol','date']],df_final['debtRatios'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['ratios'].apply(pd.Series)], axis=1)
            df = pd.concat([df[['symbol','date']],df['debtRatios'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

#Cashflow ratios
def cashflow_ratios(companies):
    link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['ratios'].apply(pd.Series)], axis=1)
    df_final = pd.concat([df_final[['symbol','date']],df_final['cashFlowIndicatorRatios'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financial-ratios/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['ratios'].apply(pd.Series)], axis=1)
            df = pd.concat([df[['symbol','date']],df['cashFlowIndicatorRatios'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

In [10]:
display(valuation_ratios(companies).head())
display(profitability_ratios(companies).head())
display(operating_ratios(companies).head())
display(liquidity_ratios(companies).head())
display(debt_ratios(companies).head())
display(cashflow_ratios(companies).head())

,symbol,date,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,receivablesTurnover,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,AAPL,2019-09-28,10.666314697418441,10.666314697418441,3.709723048190826,17.46730643441436,11.34842536857716,16.38775951405868,13.909202696891526,13.909202696891526,1.3549431647588217,3.709723048190826,0.014628458229615355,13.31637596061561,10.666314697418441
1,AAPL,2018-09-29,10.40727679085742,10.40727679085742,4.198529664752725,18.731559797584453,11.45497282843095,17.390690823755087,14.400760470981739,14.400760470981739,1.5592202481912725,4.198529664752725,0.012296561427287054,14.568458653439444,10.40727679085742
2,AAPL,2017-09-30,6.383042425417951,6.383042425417951,3.732551401624541,17.69617356414552,12.824997202640708,16.842070113969648,13.453688606559954,13.453688606559954,1.5508763494339035,3.732551401624541,0.014923546980868624,13.133266499769235,6.383042425417951
3,AAPL,2016-09-24,4.588699384790524,4.588699384790524,2.729080117232968,12.88104072055508,13.687888790148534,11.257481586196343,8.940448884905202,8.940448884905202,1.1837505702042153,2.729080117232968,0.02064584599153799,9.17266808546839,4.588699384790524
4,AAPL,2015-09-26,5.443226503456076,5.443226503456076,2.7797800711122522,12.167589978649286,13.871149623123033,9.310617950070222,7.9944417015726135,7.9944417015726135,1.1759464430725228,2.7797800711122522,0.01779501578262992,8.298511272321699,5.443226503456076


,symbol,date,niperEBT,ebtperEBIT,ebitperRevenue,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,nIperEBT,eBTperEBIT,eBITperRevenue
0,AAPL,2019-09-28,0.8405616319576494,1.0282652901611138,0.24572017188496928,0.3781776810903472,0.24572017188496928,0.2526655238417367,0.21238094505984456,0.1594383680423506,0.16323009842961633,0.6106445053487756,0.2746157613037913,0.8405616319576494,1.0282652901611138,0.24572017188496928
1,AAPL,2018-09-29,0.8165781929413056,1.0282800643177523,0.26694026619477024,0.38343718820007905,0.26694026619477024,0.27448935409175623,0.22414202074587247,0.18342180705869443,0.16277530931710985,0.5556011834209077,0.2848922482208801,0.8165781929413056,1.0282800643177523,0.26694026619477024
2,AAPL,2017-09-30,0.7544352384964659,1.0447476525821595,0.2676042820872994,0.38469860491899105,0.2676042820872994,0.27957894553164014,0.21092420845075338,0.24556476150353415,0.12882641166580963,0.3607018433832909,0.22347133931986668,0.7544352384964659,1.0447476525821595,0.2676042820872994
3,AAPL,2016-09-24,0.7444274261878381,1.0224576835932293,0.27835410106706115,0.3907595564809705,0.27835410106706115,0.284605289395703,0.211867983064288,0.2555725738121619,0.14202358821956815,0.3562366958026963,0.24733805834844239,0.7444274261878381,1.0224576835932293,0.27835410106706115
4,AAPL,2015-09-26,0.7363166241467283,1.0180401516215078,0.3047729071732666,0.4005990201741437,0.3047729071732666,0.31027105662880006,0.22845773698735639,0.26368337585327173,0.1838136319665105,0.44735453060198566,0.3394021985143113,0.7363166241467283,1.0180401516215078,0.3047729071732666


,symbol,date,receivablesTurnover,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover
0,AAPL,2019-09-28,11.34842536857716,3.4990483605848257,39.40136385776912,6.960618545668575,0.7685722388306608
1,AAPL,2018-09-29,11.45497282843095,2.9300744345834526,41.3943377148635,6.430248886306411,0.7262150522933899
2,AAPL,2017-09-30,12.824997202640708,2.8756549572876104,29.05211122554068,6.785483823224699,0.6107711040474902
3,AAPL,2016-09-24,13.687888790148534,3.5227114281117604,61.62101313320826,7.9836727138097,0.670340021014281
4,AAPL,2015-09-26,13.871149623123033,3.947280924204001,59.63771817794807,10.400738729918562,0.8045848409007191


,symbol,date,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle
0,AAPL,2019-09-28,1.540125617208044,1.168041393140241,0.46202160464632325,32.16305241876591,9.263638723714628,41.426691142480536,104.31407696777144,-72.15102454900554
1,AAPL,2018-09-29,1.1238426916297297,0.765723135899235,0.22173258261598755,31.86389050998701,8.817631109699796,40.681521619686805,124.57021422115831,-92.70632371117131
2,AAPL,2017-09-30,1.2760628484139107,0.9131172257821334,0.2012518102644474,28.46004519399391,12.563630820713515,41.023676014707426,126.92760620497985,-98.46756101098595
3,AAPL,2016-09-24,1.352669417512594,1.049401311292813,0.2592714477381465,26.665909227922594,5.923304104250396,32.58921333217299,103.61336926074777,-76.94746003282518
4,AAPL,2015-09-26,1.1087706239920605,0.7250961419178762,0.2620022329735765,26.313608454741885,6.120287817030602,32.43389627177249,92.46871631605622,-66.15510786131433


,symbol,date,debtRatio,debtEquityRatio,longtermDebtToCapitalization,totalDebtToCapitalization,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier
0,AAPL,2019-09-28,0.7326921031797611,2.7410043320661304,0.5036177624180587,9.180700000005328E10,17.87751677852349,0.6798573486043481,3.7410043320661304
1,AAPL,2018-09-29,0.7070285050242668,2.4133013523477094,0.46661721806831874,9.37350000000419E10,21.882098765432097,0.7553136491772257,3.4133013523477094
2,AAPL,2017-09-30,0.6428451530564666,1.7999060031183092,0.42034732372196804,9.720700000002733E10,26.40723202755058,0.6132705900504325,2.799906003118309
3,AAPL,2016-09-24,0.6013224075651412,1.5082924623193943,0.37032836465759345,7.542700000001689E10,41.22527472527472,0.8339858350120998,2.5082924623193943
4,AAPL,2015-09-26,0.5891097118896719,1.433739684135562,0.30936013609693436,5.346300000001426E10,97.17598908594816,1.4521380197630578,2.433739684135562


,symbol,date,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,receivablesTurnover,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatios,dividendpaidAndCapexCoverageRatios,dividendPayoutRatio
0,AAPL,2019-09-28,15.026741974700693,12.75403143551717,21.775793586343728,0.25551976255972203,11.34842536857716,0.2667099710193947,0.848755602311539,0.6798573486043481,6.7632553606237815,6.611815150071463,2.8191679531973675,0.25551976255972203
1,AAPL,2018-09-29,15.626258103066629,12.939681481348442,13.379607646401071,0.23033377567989788,11.45497282843095,0.29154916319960844,0.8280729395356046,0.7553136491772257,8.815346083788707,5.816420040561857,2.86527289546716,0.23033377567989785
2,AAPL,2017-09-30,12.18996550284614,9.737520322039883,14.218431884125751,0.264089677566131,12.824997202640708,0.27743702941099485,0.7988144281266707,0.6132705900504325,9.79033251231527,4.9705353653771,2.4877953371929276,0.26408967756613105
3,AAPL,2016-09-24,12.031834350243656,9.55542313583704,12.2751251183552,0.26593998292730975,13.687888790148534,0.30525090544845784,0.79417841516772,0.8339858350120998,18.806857142857144,4.858576911721287,2.5614444703868005,0.26593998292730975
4,AAPL,2015-09-26,14.124813741250641,12.128088662380174,7.230654596630422,0.2165224557066337,13.871149623123033,0.3477140962283123,0.8586370683926857,1.4521380197630578,32.5064,7.0739902506963785,3.5257928760466832,0.21652245570663367


# Enterprise value

In [11]:
# Company key metrics
def enterprise_value_annual(companies):
    link = ("https://financialmodelingprep.com/api/v3/enterprise-value/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['enterpriseValues'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/enterprise-value/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['enterpriseValues'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

In [12]:
enterprise_value_annual(companies).head()

,symbol,date,Stock Price,Number of Shares,Market Capitalization,- Cash & Cash Equivalents,+ Total Debt,Enterprise Value
0,AAPL,2019-09-28,209.01,4.617834E9,9.651735e+11,4.884400e+10,1.020670e+11,1.018396e+12
1,AAPL,2018-09-29,225.03,4.955377E9,1.115108e+12,2.591300e+10,1.025190e+11,1.191714e+12
2,AAPL,2017-09-30,164.0,5.217242E9,8.556277e+11,2.028900e+10,1.037030e+11,9.390417e+11
3,AAPL,2016-09-24,107.57,5.47082E9,5.884961e+11,2.048400e+10,7.892700e+10,6.469391e+11
4,AAPL,2015-09-26,112.92,5.753421E9,6.496763e+11,2.112000e+10,5.596300e+10,6.845193e+11


# Financial growth

In [13]:
def financial_growth_annual(companies):
    link = ("https://financialmodelingprep.com/api/v3/financial-statement-growth/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['growth'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/financial-statement-growth/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['growth'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

In [14]:
financial_growth_annual(companies).head()

,symbol,date,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth
0,AAPL,2019-09-28,-0.0338475436719,-0.0982820389856,-0.0982820389856,-0.0718113251919,-0.00333055786844,-0.0016792611251,-0.0681165126286,-0.0702376688188,0.104947178797,-0.103869101428,-0.0814865644641,-0.214410682333,0.875696939222,0.429389849486,-0.265058575797,0.531430680295,0.248915297308,-0.223405904741,0.843043142097,0.432850602069,-0.627101501613,0.0690454827642,-0.164106186842,0,0.672353065672,0.376707787953,-0.0112136634176,0.0379170879676,-0.0743974297628,0,-0.00440893883085,0.139154256814,0.0921879676743
1,AAPL,2018-09-29,0.154820493049,0.155744653104,0.155744653104,0.231225827801,0.295577130529,0.293159609121,-0.0501922280009,-0.0479051322888,0.130597962977,0.217554011132,0.262149873039,-0.252668674882,-0.709816421604,0.319419084921,-0.235736901281,-0.730565701994,0.106298347668,-0.220314540133,-0.699857298271,0.294494394476,-0.58852706235,-0.838057368403,0.0422906744767,0,-0.757622208903,0.377066320919,0.29719145127,-0.185169927909,-0.0255622550417,0,-0.0114172203311,0.229254813919,0.0946202739008
2,AAPL,2017-09-30,0.0465566144096,0.0219912035186,0.0219912035186,0.0583098036641,0.110179640719,0.108303249097,-0.0463510040542,-0.0451956909111,0.102026538617,-0.0338174526009,-0.0281773662866,-0.38735619935,-0.737560878465,0.46276696471,-0.403802193268,-0.775928127036,0.242324330465,-0.398525199216,-0.792407365864,0.427443259609,-0.511061676343,-0.796816252624,0.401714698551,0,-0.0804134823086,0.338684560685,0.13456899835,1.27720450281,0.166724694267,0,0.313910322196,0.152911896466,0.0751726081443
3,AAPL,2016-09-24,-0.100004272318,-0.157321353362,-0.157321353362,-0.144342060906,-0.100215517241,-0.0986984815618,-0.0491187764636,-0.0505410862532,0.105234937748,-0.190017965693,-0.250824041962,-0.450402339188,-0.663454174053,-0.78659503418,-0.411536214036,-0.703681975196,-0.792542218329,-0.458007628892,-0.702240979242,-0.791358101498,-0.553892501037,-0.717198978425,-0.824426309128,0,0,-0.80546731934,-0.0649890201199,-0.0923797360579,0.107432895321,0,0.410342547755,0.245196479484,-0.00942145299742
4,AAPL,2015-09-26,0.327331754965,0.35668437994,0.35668437994,0.351404707669,0.429892141757,0.429457364341,-0.0545800789145,-0.0538318048862,0.0990857988381,0.360943178202,0.398356713427,-0.433590635713,-0.433590635713,-0.757366391637,-0.309170893792,-0.309170893792,-0.740362393189,-0.39769112698,-0.39769112698,-0.792119494698,-0.605222470347,-0.605222470347,-0.835945815712,0,0,-0.245001763612,-0.0349942726231,0.112742775936,0.252934148267,0,0.585578693866,0.335374937924,0.194780288502


# Company key metrics

In [15]:
# Company key metrics
def key_metrics_annual(companies):
    link = ("https://financialmodelingprep.com/api/v3/company-key-metrics/{0}".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['metrics'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/company-key-metrics/{0}".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['metrics'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

def key_metrics_quarter(companies):
    link = ("https://financialmodelingprep.com/api/v3/company-key-metrics/{0}?period=quarter".format(companies[0]))
    df_final = pd.DataFrame.from_dict(get_jsonparsed_data(link))
    df_final = pd.concat([df_final['symbol'],df_final['metrics'].apply(pd.Series)], axis=1)
    for i in range (1, len(companies)):
        link = ("https://financialmodelingprep.com/api/v3/company-key-metrics/{0}?period=quarter".format(companies[i]))
        df = pd.DataFrame.from_dict(get_jsonparsed_data(link))
        if df.shape[0]!=0:
            df = pd.concat([df['symbol'],df['metrics'].apply(pd.Series)], axis=1)
            df_final = pd.concat([df_final, df])
    return df_final

In [16]:
key_metrics_annual(companies).head()

,symbol,date,Revenue per Share,Net Income per Share,Operating Cash Flow per Share,Free Cash Flow per Share,Cash per Share,Book Value per Share,Tangible Book Value per Share,Shareholders Equity per Share,Interest Debt per Share,Market Cap,Enterprise Value,PE ratio,Price to Sales Ratio,POCF ratio,PFCF ratio,PB ratio,PTB ratio,EV to Sales,Enterprise Value over EBITDA,EV to Operating cash flow,EV to Free cash flow,Earnings Yield,Free Cash Flow Yield,Debt to Equity,Debt to Assets,Net Debt to EBITDA,Current ratio,Interest Coverage,Income Quality,Dividend Yield,Payout Ratio,SG&A to Revenue,R&D to Revenue,Intangibles to Total Assets,Capex to Operating Cash Flow,Capex to Revenue,Capex to Depreciation,Stock-based compensation to Revenue,Graham Number,ROIC,Return on Tangible Assets,Graham Net-Net,Working Capital,Tangible Asset Value,Net Current Asset Value,Invested Capital,Average Receivables,Average Payables,Average Inventory,Days Sales Outstanding,Days Payables Outstanding,Days of Inventory on Hand,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share
0,AAPL,2019-09-28,56.34113309400035,11.965783092246278,15.026741974700693,12.75403143551717,10.577253318330628,19.59533408953202,73.30622971722241,19.59533408953202,22.87717574949641,9.6517348434E11,1.01839648434E12,17.46730643441436,3.709723048190826,13.909202696891526,16.38775951405868,10.666314697418441,10.666314697418441,3.914289991851607,13.31637596061561,14.676204181233878,17.291437183170334,0.05724981145517573,0.06102115418170026,2.7410043320661304,0.7326921031797611,0.695934725473018,1.540125617208044,17.87751677852349,1.2558093238743304,0.014628458229615355,0.25551976255972203,0.07012614634821313,0.07012614634821313,0.0,6.611815150071463,24.790281086231538,1.1955216769890424,0.0233228531674956,72.63369837786993,0.2746157613037913,0.16323009842961633,-27.767022374559154,5.7101E10,338516000000.0,-8.5209E10,292001000000.0,2.3056E10,5.1062E10,4.031E9,32.16305241876591,104.31407696777144,9.263638723714628,11.34842536857716,3.4990483605848257,39.40136385776912,0.6106445053487756,2.272710539183522
1,AAPL,2018-09-29,53.59733477392336,12.013414922820202,15.626258103066629,12.939681481348442,5.229269135325122,21.6223710123367,73.80366821737276,21.6223710123367,21.34227123385365,1.11510848631E12,1.19171448631E12,18.731559797584453,4.198529664752725,14.400760470981739,17.390690823755087,10.40727679085742,10.40727679085742,4.486961299384401,14.568458653439444,15.390067493736602,18.585400825158683,0.05338583710092077,0.05750202853552167,2.4133013523477094,0.7070285050242668,0.9364922189215291,1.1238426916297297,21.882098765432097,1.300734071324184,0.012296561427287054,0.23033377567989788,0.06289651537114781,0.06289651537114781,0.0,5.816420040561857,19.950048824457298,0.8189739352512582,0.020105800184491426,76.44976506303414,0.2848922482208801,0.16277530931710985,-34.89330882393005,1.4473E10,365725000000.0,-1.27239E11,338366000000.0,2.3186E10,5.5888E10,3.956E9,31.86389050998701,124.57021422115831,8.817631109699796,11.45497282843095,2.9300744345834526,41.3943377148635,0.5556011834209077,2.6865766217181863
2,AAPL,2017-09-30,43.93777401929985,9.267540206108897,12.18996550284614,9.737520322039883,3.8888362855317045,25.693076916884436,70.401948002412,25.693076916884436,20.32223155452632,8.55627688E11,9.39041688E11,17.69617356414552,3.732551401624541,13.453688606559954,16.842070113969648,6.383042425417951,6.383042425417951,4.0964328502752645,13.133266499769235,14.765270731784017,18.483981024742633,0.05650939150066401,0.05937512391487733,1.7999060031183092,0.6428451530564666,1.166613054362876,1.2760628484139107,26.40723202755058,1.3153399102397054,0.014923546980868624,0.264089677566131,0.06657389392498494,0.06657389392498494,0.02135516720443143,4.9705353653771,17.915904650254006,0.7938257131692067,0.021113796382735545,73.19502391066067,0.22347133931986668,0.131637553634047,-28.99194631953051,2.7831E10,1.26032E11,-1.12627E11,369896000000.0,2.053E10,5.24685E10,4.4055E9,28.4600451

#  Global metrics

In [17]:
sp_tickers = pd.read_csv("./data/sp_500_constituents.csv")
display(sp_tickers.head())
sp_tickers.shape[0]
companies = sp_tickers['Symbol'].values.tolist()
#companies = companies[0:3]
companies

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'AYI',
 'ADBE',
 'AAP',
 'AMD',
 'AES',
 'AET',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'APC',
 'ADI',
 'ANDV',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'AJG',
 'AIZ',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BHGE',
 'BLL',
 'BAC',
 'BAX',
 'BBT',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BHF',
 'BMY',
 'AVGO',
 'BF.B',
 'CHRW',
 'CA',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CBS',
 'CELG',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL'

# Downloaded companies

In [18]:
#companies = ["AMZN","NFLX","MSFT","AAPL","ADBE","CRM","ORCL","CSCO","IBM","FB","GOOGL",'UTX',
            #"AYX","TTD","ROKU","TLRA","ANET","SHOP","SQ","PAYC","NOW","TWTR","DBX",
            #"PYPL","V","MA","DPZ","CMG",
            #"PD","DIS","JNJ","MC","WMT","KO","XOM","SBUX","JPM","HD","LOW","TGT","COST","ADP","GD","PEP","CL","K","GIS","KMB",
            #"F","MAR","LUV","DAL","WEN","YUM",'XRX','TXN','NVDA','INTC','AMD','HPQ',
            #'PFE','BMY','MRK','ABMD','VEEV','UNH','CI','CNC','HUM','ANTM','JNJ',
            #'LB','BB','NOK','KODK','GPS','BA','HOG','MO','GE','M','OXY','APA','MRO','SLB','CVX']
    
#big_tech = ["AMZN","NFLX","MSFT","AAPL","ADBE","CRM","ORCL","CSCO","IBM","FB","GOOGL",'UTX',"PYPL","V","MA"]

#small_tech = ["AYX","TTD","ROKU","TLRA","ANET","SHOP","SQ","PAYC","NOW","TWTR","DBX"]

#stalwarts = ["PG","DIS","JNJ","MC","WMT","KO","XOM","SBUX","JPM","HD","LOW","TGT","COST","ADP","GD","PEP","CL","K","GIS","KMB"]

#cyclicals = ["F","MAR","LUV","DAL","WEN","YUM","DPZ","CMG",'AL','AIR']

#computer_hardware = ['XRX','TXN','NVDA','INTC','AMD','HPQ']

#healthcare_farmaceuticals = ['PFE','BMY','MRK','ABMD','VEEV','UNH','CI','CNC','HUM','ANTM','JNJ']

#bad_looking = ['LB','BB','NOK','KODK','GPS','BA','HOG','MO','GE','M']

#oil = ['OXY','APA','MRO','SLB','CVX']

#companies = ['AL','AIR']

# Download

In [19]:
#Descargamos todas las métricas
df_metricas = profile(companies).merge(income_annual(companies)).merge(balance_annual(companies),on=['symbol','date']).\
            merge(cashflow_annual(companies),on=['symbol','date']).\
            merge(valuation_ratios(companies),on=['symbol','date']).\
            merge(profitability_ratios(companies),on=['symbol','date']).\
            merge(operating_ratios(companies),on=['symbol','date']).\
            merge(liquidity_ratios(companies),on=['symbol','date']).\
            merge(debt_ratios(companies),on=['symbol','date']).\
            merge(cashflow_ratios(companies),on=['symbol','date']).\
            merge(enterprise_value_annual(companies),on=['symbol','date']).\
            merge(financial_growth_annual(companies),on=['symbol','date']).\
            merge(key_metrics_annual(companies),on=['symbol','date'])

#Sustituimos por NaN los valores que son none, null y los empty
df_metricas = df_metricas.replace([None],"NaN").replace('null',"NaN").replace("","NaN")


df_metricas[lista_columnas_float] = df_metricas[lista_columnas_float].astype(float)

#Cambios de nombre a columnas
df_metricas = df_metricas.rename(columns={"mktCap":"Market_cap_now","price":"Price_now","Enterprise Value_x":"EV",
                                         "Cash and short-term investments":"Total_cash",
                                         "Total debt":"Total_debt",
                                         "EV to Sales":"EV/Revenue",
                                         "Enterprise Value over EBITDA":"EV/EBITDA",
                                         "returnOnAssets":"ROA",
                                         })

#Creación de variables
df_metricas.index = df_metricas['symbol']
df_metricas['Net_cash'] = df_metricas['Total_cash'] - df_metricas['Total_debt']
df_metricas['EV'] = df_metricas['Market Capitalization'] - df_metricas['Total_cash'] + df_metricas['+ Total Debt']
df_metricas['EV/Revenue']=df_metricas['EV']/df_metricas['Revenue']
df_metricas['EV/EBITDA'] = df_metricas['EV']/df_metricas['EBITDA']
df_metricas['EV_now'] = df_metricas['Market_cap_now'] - df_metricas['Total_cash'] + df_metricas['+ Total Debt']
df_metricas['priceToSalesRatio_now'] = df_metricas['Market_cap_now']/df_metricas['Revenue']
df_metricas['priceToBookRatio_now'] = df_metricas['Market_cap_now']/(df_metricas['Total assets']-df_metricas['Total shareholders equity'])
df_metricas['priceEarningsRatio_now'] = df_metricas['Market_cap_now']/df_metricas['Net Income']
df_metricas['priceToFreeCashFlowsRatio_now'] = df_metricas['Market_cap_now']/df_metricas['Free Cash Flow']
df_metricas['EV_now/EBITDA'] = df_metricas['EV_now']/df_metricas['EBITDA']
df_metricas['Debt/EBITDA'] = df_metricas['Total_debt']/df_metricas['EBITDA']
df_metricas['Net_cash/share'] =df_metricas['Net_cash']/df_metricas['Number of Shares']

df_metricas[['grossProfitMargin','operatingProfitMargin','netProfitMargin','Free Cash Flow margin',
           'ROA','ROE','ROIC','Revenue Growth','Gross Profit Growth','EPS Growth']] = df_metricas[['grossProfitMargin','operatingProfitMargin','netProfitMargin','Free Cash Flow margin',
                                                                                      'ROA','ROE','ROIC','Revenue Growth','Gross Profit Growth','EPS Growth']]*100

save_obj(df_metricas,"df_metricas_sp500")

URLError: <urlopen error [Errno 2] No such file or directory>